In [3]:
import os
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["DATA_ROOT"] = "/home/zzy/robot/data/diffusion_policy_data/data"
os.environ["R3M_HOME"] = "/home/zzy/robot/data/.r3m"
os.environ["HF_HOME"] = "/home/zzy/robot/data/.huggingface"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["ZZY_DEBUG"] = "True"
import torch
from pathlib import Path
import numpy as np
from diffusion_policy.model.vision.r3m import R3MImageEncoderWrapper
from diffusion_policy.dataset.robomimic_replay_image_dataset import RobomimicReplayImageLanguageDataset
from torch.utils.data import ConcatDataset, DataLoader
from tqdm.auto import tqdm
from loguru import logger
import zzy_utils
model = R3MImageEncoderWrapper()
dataset_path = "/home/zzy/robot/data/diffusion_policy_data/data/robomimic/datasets/lift/ph/image_abs.hdf5"
dataset_root = Path("/home/zzy/robot/data/diffusion_policy_data/data/robomimic/datasets")
for task in ['lift', 'can', 'square']:
    for data_type in ['ph']:
        dataset_path =(dataset_root / task / data_type / 'image_abs.hdf5').as_posix()
        dataset = RobomimicReplayImageLanguageDataset(
                dataset_path=dataset_path,
                # include_tasks=['lift', 'can', 'square'],
                # data_type='ph',
                shape_meta= {
                    'obs': {
                        'agentview_image': {
                            'shape': [3, 84, 84],
                            'type': 'rgb'
                            },
                            'robot0_eye_in_hand_image': {
                                'shape': [3, 84, 84],
                                'type': 'rgb'
                            },
                            'robot0_eef_pos': {
                                'shape': [3]
                                # type default: low_dim
                            },
                            'robot0_eef_quat': {
                                'shape': [4]
                            },
                            'robot0_gripper_qpos': {
                                'shape': [2]
                            }
                        },
                        'action': {
                            'shape': [10]
                        }
                        },
                horizon=16,
                pad_before=0,
                pad_after=15,
                n_obs_steps=1,
                abs_action=True,
                rotation_rep='rotation_6d',
                use_legacy_normalizer=False,
                use_cache=False,
                seed=42,
                val_ratio=0.00,
                load_image_features=False
                )

    # break

        whole_dataset = ConcatDataset([dataset, dataset.get_validation_dataset()])
        dataloader = DataLoader(whole_dataset, batch_size=128, num_workers=0, shuffle=False)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        features = []
        actions = []
        # logger.info("start")
        for data in tqdm(dataloader):
            # print(data)
            # zzy_utils.pretty_print(data)
            features.append(model(data['obs']['image'].to(device)))
            actions.append(data['action'].to(device))
            # break
            # print(features.shape)
            # break
        features = torch.cat(features, dim=0)
        actions = torch.cat(actions, dim=0)
        print(features.shape, actions.shape)

        save_path = Path(dataset_path).absolute().as_posix().removesuffix('.hdf5') + '_features.npy'
        np.save(save_path, features.cpu().numpy())
        save_path = Path(dataset_path).absolute().as_posix().removesuffix('.hdf5') + '_actions.npy'
        np.save(save_path, actions.cpu().numpy())

# save_path = Path(dataset_path).absolute().as_posix().removesuffix('.hdf5') + '_features.npy'
# loaded = torch.load(save_path)
# print(torch.allclose(loaded, features))


100%|██████████| 76/76 [00:21<00:00,  3.58it/s]


torch.Size([9666, 512]) torch.Size([9666, 16, 10])


100%|██████████| 182/182 [01:21<00:00,  2.24it/s]


torch.Size([23207, 512]) torch.Size([23207, 16, 10])


100%|██████████| 236/236 [01:20<00:00,  2.94it/s]


torch.Size([30154, 512]) torch.Size([30154, 16, 10])


: 